In [1]:
# !wget http://dict.ruslang.ru/Freq2011.zip -O ../data/dictionaries/Freq2011.zip
# !unzip ../data/dictionaries/Freq2011.zip && mv freqrnc* ../data/dictionaries/ && rm ../data/dictionaries/Freq2011.zip

In [2]:
import re
import os
import glob
import json
import numpy as np
import pandas as pd

In [3]:
TEST_FOLDER = "/root/shared/ai-journey-2019/data/check/"

In [4]:
tests_list = glob.glob(os.path.join(TEST_FOLDER, "*.json"))

In [5]:
tests = []
tests_dfs = []

for test_path in tests_list:
    test_ = json.load(open(test_path))
    tests.append(test_)
    tests_dfs.append(pd.DataFrame.from_dict(test_['tasks']))

In [6]:
df_tests = pd.concat(tests_dfs)

In [7]:
df_dict = pd.read_table("../data/dictionaries/freqrnc2011.csv")

# Analysis:

# 11-12 questions:

In [8]:
# !wget https://raw.githubusercontent.com/danakt/russian-words/master/russian.txt -O ../data/dictionaries/russian.txt

In [9]:
encoding = "windows-1251"

In [17]:
%%time

df_dict_full = pd.read_csv("../data/dictionaries/russian_1.5kk_words.txt", encoding=encoding, header=None)
df_dict_full.columns = ["Lemma"]
big_words_set = frozenset(df_dict_full["Lemma"].values)

CPU times: user 3.41 s, sys: 244 ms, total: 3.66 s
Wall time: 3.65 s


In [18]:
df_dict = pd.read_table("../data/dictionaries/freqrnc2011.csv")
small_words_dict = df_dict.set_index("Lemma")[["Freq(ipm)"]].to_dict()["Freq(ipm)"]

In [19]:
def remove_additional(word):
    additional_words = re.findall("\([\w\ ]+\)", word)

    for additional_word in additional_words:
        word = word.replace(additional_word, "")
        word = word.strip()
        
    return word

In [20]:
def check_pair(word_1, word_2):
    # TODO: EMBEDDINGS NEEDED TO USE NOT ONLY WORD BUT WITH CONTEXT
    word_1 = remove_additional(word_1)
    word_2 = remove_additional(word_2)
    russian_letters = list("абвгдеёжзийклмнопрстуфхцчшщъыьэюя") + [""]
    letters_1 = []
    letters_2 = []
    for letter_ in russian_letters:
        if word_1.replace("..", letter_) in big_words_set:
            letters_1.append(letter_)
#     print(letters_1)
    for letter_ in letters_1:
        if word_2.replace("..", letter_) in big_words_set:
            letters_2.append(letter_)
#     print(letters_2)
    return list(set(letters_2).intersection(letters_1))

In [41]:
def solver_11_12(task, df_dict):
    if "Выпишите слово" in task["text"]:
        bukva_list = re.findall("буква [А-Яа-я]", task["text"])
    
        if len(bukva_list):
            bukva = bukva_list[-1][-1].lower()
            words_list = task["text"].replace("..", bukva).split("\n")[1:]
            conf = 0.
            most_conf_word = None
            for word_ in words_list:
                word_ = remove_additional(word_)
                temp_conf = small_words_dict.get(word_, -1.0)
                if temp_conf > conf:
                    conf = temp_conf
                    most_conf_word = word_
            if conf == 0.:
                for word_ in words_list:
                    word_ = remove_additional(word_)
                    words_ = big_words_set.intersection({word_})
                    if len(words_):
                        conf = 1.0
                        most_conf_word = word_
                   
            print(most_conf_word, task['solution']['correct'])
            
    elif "пропущена одна и та же буква":
        ids = []
        for answer_ in task["question"]["choices"]:
            if len(check_pair(*answer_["text"].split(", "))):
                ids.append(answer_['id'])
        
        print(ids, task["solution"])
        
    else:
        print("Unknown question type")

In [42]:
%%time

for test_ in tests:
    task_ = test_['tasks'][10]
    
    solver_11_12(task_, df_dict)
    print()

затмевает затмевает

['2', '4', '5'] {'correct_variants': [['2', '4'], ['4', '2']]}

заведовать заведовать

болевой болевой

['3', '5'] {'correct_variants': [['3', '5'], ['5', '3']]}

['2', '3', '5'] {'correct': ['2', '5']}

рыжеватый рыжеватый

['2', '4', '5'] {'correct': ['2', '4', '5']}

вишенка вишенка

приказчик приказчик

CPU times: user 0 ns, sys: 2.36 ms, total: 2.36 ms
Wall time: 1.98 ms


In [43]:
%%time

for test_ in tests:
    task_ = test_['tasks'][11]
    
    solver_11_12(task_, df_dict)
    print()

мыслящий мыслящий

['2', '4', '5'] {'correct': ['2', '4', '5']}

выветрившийся выветрившийся

заклеишь заклеишь

['1', '2', '3', '5'] {'correct': ['1', '3', '5']}

['1', '4'] {'correct': ['1', '4']}

неотъемлемый неотъемлемый

['3', '4'] {'correct_variants': [['3', '4'], ['4', '3']]}

мелют мелют

видимый видимый

CPU times: user 2.49 ms, sys: 689 µs, total: 3.17 ms
Wall time: 2.28 ms
